# 9. GLMのベイズモデル化と事後分布の推定
* 複数のパラメーターをもつGLMのベイズモデル化、そしてそのパラメーターの事後分布の推定方法について扱う

#### 滝沢補足
* ベイズは統計学の考え方の1つ。統計モデルを推定する道具。([参考](http://logics-of-blue.com/%E3%83%99%E3%82%A4%E3%82%BA%E3%81%A8%E7%B5%B1%E8%A8%88%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E9%96%A2%E4%BF%82/))

## 9.1. 例題：種子数のポアソン回帰（個体差なし）

* 例題の目的
    * 個体ごとの平均種子数が、体サイズ$x_i$にどう依存しているのか調べる
* まずはこの例題のGLMをベイズ統計モデル化する方法について検討し、そのパラメータの事後分布の推定に取り組む。

## 9.2. GLMのベイズモデル化

* ベイズモデル化したGLMでも、モデルの中核部分はポアソン回帰のGLM。
* 個体$i$の種子数$y_i$のばらつきを平均$\lambda_i$のポアソン分布$p(y_i|\lambda_i)$に従うとする。
* 平均を下記と指定。
$$
\lambda_i = \exp(\beta_1+\beta_2 x_i)
$$
* モデルの尤度関数は下記。
$$
L(\beta_1,\beta_2)=\prod_i p(y_i|\lambda_i) = \prod_i p(y_i|\beta_1,\beta_2,x_i)
$$
* この例題でも説明変数$x_i$を定数のように扱う。(「でも」とは、どれと一緒だと言ってる？）
* 尤度関数は離散確率分布で定義されているので、（ポアソン分布の尤度だから、離散確率分布ということ？）
$$
p({\bf Y}|\beta_1,\beta_2) = L(\beta_1,\beta_2)
$$
* ベイズモデルの事後分布は(尤度)$\times$(事前分布)に比例するので、
$$
p(\beta_1,\beta_2|{\bf Y}) \propto p({\bf Y}|\beta_1,\beta_2) p(\beta_1) p(\beta_2)
$$
    * (復習) p186 8.4.より。 ベイズ統計モデルとは、下記の構造を持つ統計モデル。
        * 事後分布 = 尤度$\times$事前分布 / データが得られる確率

## 9.3. 無情報事前分布

Q. ベイズ統計モデルにおける事前分布 $p(\beta_*)$とは？

A.
* 観測データ${\bf Y}$が得られていない時のパラメータ$\beta_*$の確率分布
* わかるはずもない

---

* そこで、線形予測子のパラメータ$\beta_*$の値は[$-\infty,\infty$]の範囲で「好きな値をとって良い」といったことを表現する事前分布$p(\beta_*)$を設定する。
    * 無情報事前分布
* しかし、区間全体で密度関数を積分しても1にならない
* そこで、「無情報っぽい」事前分布として下記の2種類が用いられる。
    1. 広い範囲をとる一様分布（$-10^9 < \beta_* < 10^9$など)
    2. 平べったい正規分布
        * 平均ゼロで標準偏差がとても大きい(標準偏差100など）

## 9.4. ベイズ統計モデルの事後分布の推定

* 9.3で、パラメータの事前分布$p(\beta_*)$を無情報事前分布と決めることができた
* 次に、このベイズ統計モデルと例題データに基づいて、切片$\beta_1$と傾き$\beta_2$の事後分布 ー つまり複数の連続値パラメータの事後分布$p(\beta_1,\beta_2|{\bf Y})$をMCMCサンプリングを使って推定する。

* この本ではWinBUGSというソフトウェアを使用している。しかし、Macだと微妙そう。
    * 今はStanがコミュニティが大きいこと、またネット上にも情報が多いことから、Stanがお勧めとのこと。(TJOさんの[ツイート](https://twitter.com/TJO_datasci/status/883643447126757376))
* 具体的なコードは飛ばし、流れだけ記載します。
* Stanについては、[StanとRでベイズ統計モデリング](http://amzn.to/2uVu97t)　（通称アヒル本）が良さそうです。

### 9.4.1. ベイズ統計モデルのコーディング

* まずはベイズモデル化したGLMをBUGSコードで表現

### 9.4.2. 事後分布推定の準備

* データやパラメータの初期値、サンプリング回数の指示。
* 最初の何ステップの結果を使わないとするか。

### 9.4.3. どれだけ長くMCMCサンプリングすればいいのか

* MCMCサンプリングの結果から定常分布（この例題の場合は事後分布）を推定する。
* サンプリングする長さや値を記録する間隔に注意が必要
    1. サンプリング量が少ないと、事後分布を正確に推定できないので、量を多くすること。
    2. 複数のサンプリングの比較をすること。
        * 初期値の影響を受けてる範囲が掴める(これらは事後分布推定用の値としては望ましくない。
        * データ・統計モデル・MCMCサンプリングのどこかに問題があれば、サンプル列ごとに異なる挙動を示す。
            * サンプル列間の乖離の大小を調べることを収束診断と呼ぶ
            
* 収束診断は$\hat R$指数で表すのが手軽。
    * $\hat R$が$1.0$に近ければ収束しているとみなす
    * $\hat R>1.1$となるような時には、サンプル列間のばらつきが大きいので、定常分布・事後分布は推定できないと判断する。
        * 収束しない時には、統計モデルや、サンプリング法に何か問題がある。

## 9.5. MCMCサンプルから事後分布を推定

* 500個$\times $3ステップのサンプル列をえた。これらを図示する。
* 個々のパラメーターの事後分布の様子を把握する(図9.5)
* 図9.5(B)(D)は$\beta_1,\beta_2$それぞれの周辺事後分布
* 図9.6(A)は事後分布からのサンプルの組み合わせごとに平均$\lambda$を計算し、一本ずつ直線を引いている。
* 図9.6(B)は$\beta_1,\beta_2$の散布図。

### 9.5.1. 事後分布の統計量

* $\beta_,\beta_2$の平均、中央値、95％信用区間など。

## 9.6. 複数パラメーターのMCMCサンプリング

* MCMCアルゴリズムについて説明
* 複数パラメーターの事後分布からMCMCサンプリングする方法を考えてみる。
* MCMCサンプリングとは、値を少しずつ変更していく方式。
    * しかし、複数のパラメーターを同時に変更するのは難しい。
        * そこで、$\beta_1,\beta_2$の値を交互に更新する方法をとってみる。

### 9.6.1. ギブスサンプリング：この章の例題の場合

* 様々なアルゴリズムを使ってMCMCサンプリングする。
    * その中でもギブスサンプリングが特に重要。
* ギブスサンプリングとメトロポリス法の相違点
    * メトロポリス法：新しい値の候補をあげ、それに変化するかどうかを決める
    * ギブスサンプリング：「新しい値の確率分布」を作り、その確率分布のランダムサンプルを新しい値とする。
        * 「新しい値の確率分布」とは、多変量確率分布から一つの変量を除いて、他の変量全てを定数とする一変量確率分布。(全条件付き分布(FCD))
        * メトロポリス法と比較した利点
            * 各MCMCステップにおいて、元の値と更新された値の相関がより小さい(参考：8.3.2.マルコフ連鎖の定常分布)
            * MCMCサンプリングの詳細を指定しなくても良い
* ギブスサンプリングのアルゴリズム
    1. パラメータ($\beta_1,\beta_2$)の初期値を何か決める。
    2. 他のパラメーター($\beta_2$)を定数として、あるパラメーター($\beta_1$)のFCDを求める。
    3. このFCDに従う乱数を1つ発生させ、 パラメーター($\beta_1$)を更新する。
    4. 他のパラメーター($\beta_2$)について、同様に乱数を発生させ、更新する。
    5. ここまでで第1MCMCステップが終了。引き続き、交互にサンプリングする。

### 9.6.2. WinBUGSの挙動はどうなっている？

## 9.7. この章のまとめと参考文献

* この章の主題
    * GLMのベイズモデル化
    * 複数のパラメーターの事後分布からのMCMCサンプリング
    
* 全個体に共通するパラメーターの事前分布として、「どのような値でも構わない」ことを表現する無情報事前分布を指定する
* ベイズ統計モデル化したGLMを記述する
* MCMCサンプリングを行い、事後分布の統計量の分布を把握。MCMCサンプル列の収束診断を行う
* ギブスサンプリングについて
* ベイズ統計モデルでもモデル選択ができれば良いが、手法が定まっていない
    * 交差検証法
    * DIC